In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

if os.getcwd().endswith('notebook'):
    os.chdir('..')
print(f'current working directory: {os.getcwd()}')

current working directory: C:\Users\jibin-cmii\ntu_sd6103_data_systems_team_project-main


In [4]:
# load data
pub_csv_path = r"F:\MS\1119\parsed_data\publications.csv"
author_csv_path = r"F:\MS\1119\parsed_data\authors.csv"
author_pub_csv_path = r"F:\MS\1119\parsed_data\author_publications.csv"
jrnls_csv_path = r"F:\MS\1119\parsed_data\journals.csv"
conf_csv_path = r"F:\MS\1119\parsed_data\conferences.csv"
editors_csv_path = r"F:\MS\1119\parsed_data\editors.csv"
publisher_csv_path = r"F:\MS\1119\parsed_data\publishers.csv"
pub_df = pd.read_csv(pub_csv_path)
au_pub_df = pd.read_csv(author_pub_csv_path)
au_df = pd.read_csv(author_csv_path)
jrnls_df = pd.read_csv(jrnls_csv_path)
conf_df = pd.read_csv(conf_csv_path)
editors_df = pd.read_csv(editors_csv_path)
publisher_df = pd.read_csv(publisher_csv_path)

In [7]:
for df in [pub_df, au_pub_df, au_df, jrnls_df, conf_df, editors_df, publisher_df]:
    # print(df.info())
    #print(df.head())
    print(df.columns.tolist())
    print(df.shape)
    print('------------------------------------')

['type', 'pub_id', 'mdate', 'year', 'key_source', 'key_name', 'volume', 'publisher', 'journal', 'number', 'url', 'ee']
(11179600, 12)
------------------------------------
['author_id', 'author_name', 'pub_id']
(28556780, 3)
------------------------------------
['author_id', 'name']
(28556780, 2)
------------------------------------
['journal_id', 'name']
(3669449, 2)
------------------------------------
['conf_id', 'name']
(0, 2)
------------------------------------
['editor_id', 'name']
(152730, 2)
------------------------------------
['publisher_id', 'name']
(93944, 2)
------------------------------------


In [48]:
import pandas as pd
import re

# pub_df

# conference list
pub_df[(pub_df['key_source'] == 'conf') & (pub_df['type'] == 'proceedings')]

# 定义一个函数来转换 pub_id
def transform_pub_id(pub_id):
    # 使用正则表达式匹配并提取 dac 和 1995
    match = re.match(r'(\w+)/(\d+)', pub_id)
    if match:
        return f"{match.group(1)}{match.group(2)}"
    else:
        return pub_id  # 如果格式不符合预期，保持原样

# 应用转换函数到 pub_id 列
conf_df['conf_id'] = conf_df['key_name'].apply(transform_pub_id)
conf_df['name']=conf_df['publisher']

# 打印前几行查看结果
conf_df

conf_df['conf_id'] = range(0, len(conf_df))
conf_df_selected = conf_df[['conf_id', 'pub_id']]
conf_df_selected.rename(columns={'key_name': 'name'})
conf_df_selected.to_csv(r'F:\MS\1119\parsed_data\result\conferences.csv', index=False)
conf_df_selected

,conf_id,pub_id
58289,0,dac1995
58331,1,dac2010
58449,2,dac2004
58583,3,dac1964
58801,4,dac1986
...,...,...
3678517,59245,sgiot2021
3678554,59246,sgiot2023
3678575,59247,sgiot2020
3678581,59248,sgiot2018


In [5]:
import pandas as pd
import uuid
import re
from tqdm import tqdm
print('-1')
# 读取CSV文件
df = pd.read_csv(r'F:\MS\1119\parsed_data\author_publications.csv')
print('0')


-1


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


0


In [6]:
# 筛选出 author_id 为空的行
df_null = df[df['author_id'].isna()]

# 筛选出 author_id 不为空的行
df_not_null = df[df['author_id'].notna()]

# 保存到新的CSV文件
df_null.to_csv(r'F:\MS\1119\parsed_data\result\author_publications_null.csv', index=False)
df_not_null.to_csv(r'F:\MS\1119\parsed_data\result\author_publications_not_null.csv', index=False)

print("CSV files have been split and saved.")

CSV files have been split and saved.


In [7]:
print('-1')
# 读取CSV文件
author_df = pd.read_csv(r'F:\MS\1119\parsed_data\authors.csv')
print('0')

-1
0


In [8]:
# 筛选出 author_id 为空的行
author_df_null = author_df[df['author_id'].isna()]

# 筛选出 author_id 不为空的行
author_df_not_null = author_df[df['author_id'].notna()]

# 保存到新的CSV文件
author_df_null.to_csv(r'F:\MS\1119\parsed_data\result\author_null.csv', index=False)
author_df_not_null.to_csv(r'F:\MS\1119\parsed_data\result\author_not_null.csv', index=False)

print("CSV files have been split and saved.")

CSV files have been split and saved.


In [7]:
author_df_not_null_from_csv=pd.read_csv(r'F:\MS\1119\parsed_data\result\process\author_null.csv')
# 对 name 列进行去重
print(author_df_not_null_from_csv.shape)
df_unique = author_df_not_null_from_csv.drop_duplicates(subset=['name'])
print(df_unique.shape)

# 生成编号
def generate_id(index):
    # 计算 yyyy 和 XXXX
    xxxx = (index + 1) % 10000
    yyyy = (index // 10000) + 1
    return f"0000-0004-{str(yyyy).zfill(4)}-{str(xxxx).zfill(4)}"

# 为去重后的 name 列生成编号
df_unique['author_id'] = df_unique.reset_index(drop=True).index.map(generate_id)

# 保存处理后的CSV文件
df_unique.to_csv(r'F:\MS\1119\parsed_data\result\process\1author_publications_unique.csv', index=False)

print("CSV file has been processed and saved.")

(22806838, 2)
(3630718, 2)


<ipython-input-7-babd4c730b30>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['author_id'] = df_unique.reset_index(drop=True).index.map(generate_id)


CSV file has been processed and saved.


In [13]:
import pandas as pd
# 读取CSV文件
df1 = pd.read_csv(r'F:\MS\1119\parsed_data\result\process\1author_unique.csv')
df2 = pd.read_csv(r'F:\MS\1119\parsed_data\result\process\author_publications_null.csv')


In [15]:

# 将df1保存到字典中，key是name，value是author_id
author_id_map = dict(zip(df1['name'], df1['author_id']))

# 遍历df2，更新author_id列
df2['author_id'] = df2['author_name'].map(author_id_map)

# 保存处理后的CSV文件
df2.to_csv(r'F:\MS\1119\parsed_data\result\process\2author_publications_null_updated.csv', index=False)

print("author_publications_null_updated CSV file has been processed and saved.")

author_publications_null_updated CSV file has been processed and saved.


In [17]:
import pandas as pd

# 读取CSV文件
df1 = pd.read_csv(r'F:\MS\1119\parsed_data\result\process\1author_not_null.csv')
df2 = pd.read_csv(r'F:\MS\1119\parsed_data\result\process\1author_unique.csv')

# 合并两个DataFrame
combined_df = pd.concat([df1, df2], ignore_index=True)

# 去重：所有列的值都相等的行
unique_df = combined_df.drop_duplicates()

# 保存处理后的CSV文件
unique_df.to_csv(r'F:\MS\1119\parsed_data\result\authors.csv', index=False)

print("CSV file has 合并 been processed and saved.")

CSV file has 合并 been processed and saved.


In [18]:
import pandas as pd

# 读取CSV文件
df1 = pd.read_csv(r'F:\MS\1119\parsed_data\result\process\2author_publications_not_null.csv')
df2 = pd.read_csv(r'F:\MS\1119\parsed_data\result\process\2author_publications_null_updated.csv')

# 合并两个DataFrame
combined_df = pd.concat([df1, df2], ignore_index=True)

# 去重：所有列的值都相等的行
unique_df = combined_df.drop_duplicates()

# 保存处理后的CSV文件
unique_df.to_csv(r'F:\MS\1119\parsed_data\result\author_publications.csv', index=False)

print("CSV file has 合并 been processed and saved.")

CSV file has 合并 been processed and saved.
